In [ ]:
!pip install torch torchvision pillow matplotlib
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
from model.unet import UNet
from utils import COLOR_MAP
import numpy as np


In [ ]:
model = UNet()
model.load_state_dict(torch.load("unet_model.pth", map_location="cpu"))
model.eval()


In [ ]:
def predict(image_path, color_name):
    img = Image.open(image_path).convert("RGB")
    color_vec = COLOR_MAP[color_name.lower()]
    color_channel = np.ones((img.size[1], img.size[0], 1)) * np.array(color_vec).mean()
    img_cond = np.concatenate([np.array(img)/255.0, color_channel], axis=2)
    tensor = transforms.ToTensor()(img_cond).unsqueeze(0)
    with torch.no_grad():
        output = model(tensor)
    out_img = transforms.ToPILImage()(output.squeeze(0).clamp(0,1))
    return out_img

result = predict("dataset/validation/inputs/sample.png", "blue")
plt.imshow(result)
plt.axis("off")
plt.show()
